In [5]:
from elasticsearch import Elasticsearch
import pandas as pd

# Αν έχεις απενεργοποιήσει το security
es = Elasticsearch("http://localhost:9200")

# Έλεγχος σύνδεσης
print(es.info())

{'name': 'ANTHIPPI', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'spRS4hvpTO6ikxSrVs-Tiw', 'version': {'number': '9.1.5', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '90ee222e7e0136dd8ddbb34015538f3a00c129b7', 'build_date': '2025-10-02T22:07:12.966975992Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [4]:
import pandas as pd

file_path = '../data/documents.csv' 

try:
    df = pd.read_csv(file_path)
    print("Το αρχείο φορτώθηκε επιτυχώς!")
except FileNotFoundError:
    print(f"Πρόσοχή: Το αρχείο δεν βρέθηκε στη διαδρομή: {file_path}")

print(f"Διαστάσεις (γραμμές, στήλες): {df.shape}")
print("\nΟι πρώτες 5 γραμμές:")
print(df.head())
print("\nΚενές τιμές ανά στήλη:")
print(df.isnull().sum())

if 'text' in df.columns:
    initial_rows = len(df)
    df = df.dropna(subset=['text'])
    print(f"\nΑφαιρέθηκαν {initial_rows - len(df)} γραμμές με κενό κείμενο.")

    df['text'] = df['text'].astype(str)

if 'id' in df.columns:
    df['id'] = df['id'].astype(str)

print("\nΈτοιμο DataFrame για Indexing:")
print(df.head(3))

Το αρχείο φορτώθηκε επιτυχώς!
Διαστάσεις (γραμμές, στήλες): (18316, 2)

Οι πρώτες 5 γραμμές:
       ID                                               Text
0  193157  Support towards the Europe PMC initiative-Cont...
1  193158  Support to the Vice-Presidents of the ERC Scie...
2  193159  Implementation of activities described in the ...
3  193160  Monitoring Atmospheric Composition and Climate...
4  193161  Pre-Operational Marine Service Continuity in T...

Κενές τιμές ανά στήλη:
ID      0
Text    0
dtype: int64

Έτοιμο DataFrame για Indexing:
       ID                                               Text
0  193157  Support towards the Europe PMC initiative-Cont...
1  193158  Support to the Vice-Presidents of the ERC Scie...
2  193159  Implementation of activities described in the ...


In [6]:
index_name = "ir2025_index"

settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "default": {
                    "type": "english" 
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "doc_id": {"type": "keyword"}, 
            "text": {
                "type": "text",
                "similarity": "BM25"
            }
        }
    }
}

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ir2025_index'})

In [10]:
from elasticsearch.helpers import bulk

# Ορίζουμε το όνομα του ευρετηρίου (πρέπει να είναι ίδιο με αυτό που έφτιαξες πριν)
index_name = "ir2025_index"

# Συνάρτηση που ετοιμάζει τα δεδομένα για την ElasticSearch
def generate_data(dataframe):
    for index, row in dataframe.iterrows():
        yield {
            "_index": index_name,
            "_source": {
                # ΠΡΟΣΟΧΗ: Εδώ υποθέτω ότι οι στήλες στο CSV λέγονται 'id' και 'text'.
                # Αν στο δικό σου df.head() είδες άλλα ονόματα (π.χ. 'doc_id', 'content'),
                # άλλαξε τα παρακάτω δεξιά μέρη: row['το_δικό_σου_όνομα']
                "doc_id": str(row['ID']), 
                "text": row['Text']
            }
        }

print("Ξεκινάει η εισαγωγή των κειμένων (αυτό μπορεί να πάρει λίγη ώρα)...")

# Εκτέλεση της μαζικής εισαγωγής
# Το es είναι η μεταβλητή σύνδεσης που έφτιαξες στο προηγούμενο βήμα
success, failed = bulk(es, generate_data(df))

print(f"Ολοκληρώθηκε! Επιτυχίες: {success}, Αποτυχίες: {failed}")

# Κάνουμε refresh για να είναι τα δεδομένα άμεσα αναζητήσιμα
es.indices.refresh(index=index_name)
print("Το ευρετήριο ανανεώθηκε και είναι έτοιμο για αναζήτηση.")

Ξεκινάει η εισαγωγή των κειμένων (αυτό μπορεί να πάρει λίγη ώρα)...
Ολοκληρώθηκε! Επιτυχίες: 18316, Αποτυχίες: []
Το ευρετήριο ανανεώθηκε και είναι έτοιμο για αναζήτηση.


In [12]:
import pandas as pd

queries_path = '../data/queries.csv'
queries_df = pd.read_csv(queries_path)

queries_df.columns = queries_df.columns.str.strip()

qid_col = 'ID'
qtext_col = 'Text'

output_file = "../results/phase1_results.txt"  
run_id = "my_elastic_bm25" 

print("Ξεκινάει η αναζήτηση...")

with open(output_file, 'w') as f:
    for index, row in queries_df.iterrows():
        q_id = str(row[qid_col])
        q_text = row[qtext_col]
        
        if pd.isna(q_text):
            continue

        response = es.search(
            index="ir2025_index",
            body={
                "query": {
                    "match": {
                        "text": q_text 
                    }
                },
                "size": 50 
            }
        )

        hits = response['hits']['hits']
        
        for rank, hit in enumerate(hits):
            doc_id = hit['_source']['doc_id']
            score = hit['_score']
            
            line = f"{q_id}\tQ0\t{doc_id}\t{rank+1}\t{score:.4f}\t{run_id}\n"
            f.write(line)

print(f"Η αναζήτηση ολοκληρώθηκε! Τα αποτελέσματα είναι στο: {output_file}")

Ξεκινάει η αναζήτηση...
Η αναζήτηση ολοκληρώθηκε! Τα αποτελέσματα είναι στο: ../results/phase1_results.txt
